In [ ]:
# READ IN THE CSV

import pandas as pd
import numpy as np

df_sf = pd.read_csv('Smart_Farm_dataset.csv')

In [ ]:
# DO ANY PRE OBSERVATION HERE

df_sf

In [ ]:
# ONE HOT REPRESENT (PREVIOUSLY) STRING ENCODED LOC AND GEN COLUMNS

df_sf = pd.concat([df_sf, pd.get_dummies(df_sf['Loc'])], axis=1)
df_sf = pd.concat([df_sf, pd.get_dummies(df_sf['Gen'])], axis=1)

# DROP ANY COLUMNS HERE
COLUMNS_DROPPED = ['Date', 'Loc', 'Gen', 'PlantID']
df_sf = df_sf.drop(COLUMNS_DROPPED, axis=1)

In [ ]:
# TRAIN AND TEST SPLIT

X_train = pd.DataFrame()
y_train = np.array([])
X_test = pd.DataFrame()
y_test = np.array([])

true_index_counter = -1
for i in range (0, int(len(df_sf)/100)):
    for k in range (1, 101):
        true_index_counter += 1
        if k <= 95:
            X_train = X_train.append(df_sf.loc[true_index_counter].drop(['GrowthRate']))
            y_train = np.append(y_train, df_sf.loc[true_index_counter, 'GrowthRate'])
        else:
            X_test = X_test.append(df_sf.loc[true_index_counter].drop(['GrowthRate']))
            y_test = np.append(y_test, df_sf.loc[true_index_counter, 'GrowthRate'])


In [ ]:
# ERROR COMPUTATION FUNCTIONS

from sklearn.metrics import mean_squared_error
def evaluate_errors(prediction, actual):
    print("RMSE Error: ", np.sqrt(mean_squared_error(prediction, actual)))
    avg_error_vector = np.absolute(((preds - y_test) / y_test) * 100)
    print("Average Error details:\n", avg_error_vector.describe())
    return avg_error_vector

In [ ]:
# FEATURE IMPORTANCE FUNCTIONS

def get_feature_importances(regr):
    feature_importances = regr.feature_importances_
    feature_importances = pd.Series(feature_importances)
    feature_importance_df = pd.DataFrame({'feature': X_train.columns,'feature_importance': feature_importances})
    feature_importance_df = feature_importance_df.sort_values(by=['feature_importance'])
    for index, row in feature_importance_df.iterrows():
        print(row['feature'], 'has importance: ', row['feature_importance'])

In [ ]:
# MODELS RF

from sklearn.ensemble import RandomForestRegressor
regr = RandomForestRegressor(n_estimators=20, max_depth=13, random_state=0, verbose=1, n_jobs=-1)
regr.fit(X_train, y_train)
preds = regr.predict(X_test)

preds = pd.DataFrame(preds)
y_test = pd.DataFrame(y_test)
evaluate_errors(preds, y_test)
get_feature_importances(regr)

In [ ]:
# MODELS LIGHTGBM

from lightgbm import LGBMRegressor

clf = LGBMRegressor(n_estimators=1000, learning_rate=0.01)
clf.fit(X_train, np.log1p(y_train))
preds = np.expm1(clf.predict(X_test))

preds = pd.DataFrame(preds)
y_test = pd.DataFrame(y_test)

evaluate_errors(preds, y_test)